In [628]:
import json
import pandas as pd
import numpy as np

import requests
from flask_restx import Model
from pydantic import BaseModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

Доступные ручки:

In [629]:
info_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/task_result?task_id={0}'
train_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/train'
delete_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/delete'
predict_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/predict'

Посмотрим, как работает созданное API. Для этого запустим файл `my_api.py` и покидаем в ручки запросы. Первая ручка `info_endpoint` - с информацией о доступных моделях и гиперпараметрах

In [631]:
r = requests.get(train_endpoint).json()
print(requests.get(info_endpoint.format(r)).json())

''

Создадим пару тестовых датасетов

In [632]:
x_1, y_1 = make_classification(n_samples=400, n_features=5, n_classes=2)

dataset_train_1 = pd.DataFrame(x_1[:100, :])
dataset_train_1['target'] = y_1[:100]
dataset_train_1 = dataset_train_1.to_json()

dataset_train_2 = pd.DataFrame(x_1[100:300, :])
dataset_train_2['target'] = y_1[100:300]
dataset_train_2 = dataset_train_2.to_json()

dataset_test = pd.DataFrame(x_1[300:, :])
dataset_test = dataset_test.to_json()

Получаем из ручки `train_endpoint` информацию об обученных моделях - их нет

Отправим запрос с обучением случайного леса с заданными гиперпараметрами; в ответ получаем небольшой отчет об основных метриках обучения на 5 фолдах 

In [638]:
json_train_data = {
    'model_name': 'rfc',
    'model_type': 'Random Forest',
#     'hyperparams': {'n_estimators': 200, 'max_depth': 10},
    'train_data': dataset_train_1,
}

r = requests.post(train_endpoint, json=json_train_data)
print(r.json())

85d38ade-de9a-44f7-9b54-1647d6c8e9c2


In [639]:
requests.get(info_endpoint.format(r.json())).json()

'{"score":{"precision":0.915,"recall":0.96,"roc_auc":0.947,"f1_score":0.935}}'

Проверим, что в списке появилась обученная модель

In [654]:
r = requests.get(train_endpoint).json()
print(requests.get(info_endpoint.format(r)).json())

rfc
rfc



Отправим теперь `dataset_train_2`, на котором переобучим созданную `rfc` модель

In [648]:
json_train_data = {
    'model_name': 'rfc',
    'new_data': dataset_train_2,
}

r = requests.put(train_endpoint, data=json_train_data)
print(r.json())

e55d6ec6-8496-4797-9405-7158ab4ea03f


In [649]:
requests.get(info_endpoint.format(r.json())).json()

'{"score":{"precision":0.858,"recall":0.854,"roc_auc":0.925,"f1_score":0.85}}'

Проверим, что обученная модель работает - дадим ей на вход оставшуюся часть датасета и посмотрим на `classification_report`

In [650]:
json_pred_data = {
    'model_name': 'rfc',
    'predict_data': dataset_test,
}

r = requests.post(predict_endpoint, json=json_pred_data)
print(r.json())
# y_pred = r.json()
# print(classification_report(y_1[300:], y_pred))

de8a54d8-4d81-4a01-8a82-2696ec44851e


In [651]:
requests.get(info_endpoint.format(r.json())).json()

[1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1]

Напоследок удалим созданную модель с помощью ручки `delete_endpoint` и выведем список обученных моделей

In [652]:
r = requests.delete(delete_endpoint + '/' + 'rfc')
r.json()

'536876bf-4ff9-4207-ab57-a59e10b7bfd2'

In [563]:
requests.get(f'http://127.0.0.1:5000/test_api/ml_models/train?task_id=ea2e805b-e3d1-453f-bd7b-17d94693dfa8 rfc').json()


'PENDING'

# Тестирование mongo хранилища

In [44]:
!pip install pymongo

     |████████████████████████████████| 351 kB 1.6 MB/s            


In [393]:
from pymongo import MongoClient

In [394]:
client = MongoClient('localhost', 27017)

In [395]:
client.list_database_names()

['admin', 'api_database', 'config', 'local']

In [396]:
db = client['test_databse']

In [397]:
collection = db['test_collection']

In [398]:
test_1 = {'user1': 'password1', 'user2': 'password2'}
# test_2 = {'user3': {'password3': 'abracadabra'}, 'user4': {'password4': 'keklol'}}
test_2 = {'user1': 'password2', 'user2': 'password2'}

In [399]:
collection.insert_one(test_1)
collection.insert_one(test_2)
# collection.update_one({'user1': 'password1'}, {'$set': {'user1': 'password123'}})

In [389]:
collection.find_one({'user3': {'password3': 'abracadabra'}})
collection.find_one({'user2': 'password2'})

{'_id': ObjectId('61ae7f2f14f40384aa6509ec'),
 'user1': 'password123',
 'user2': 'password2'}

In [153]:
collection.delete_one({'user2': 'password2'})

# Тестирование объектного хранения в монге

In [280]:
import pickle

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [281]:
model = RandomForestClassifier()

X, y = make_classification(n_samples=400, n_features=5, n_classes=2)
X_train, y_train = X[:300], y[:300]
X_test, y_test = X[300:], y[300:]

model.fit(X_train, y_train)

RandomForestClassifier()

In [283]:
collection.insert_one({'model_name': 'logreg_1', 'model': pickle.dumps(model)})

In [284]:
output = collection.find_one({'model_name': 'logreg_1'})

In [142]:
pickle.loads(output['model'])

LogisticRegression()

In [287]:
import pickle

In [288]:
pickle.loads(output['model'])

RandomForestClassifier()

In [417]:
import pickle
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

class MongoDBClient:

    def __init__(self, host, port):
        self.client = MongoClient(host, port)
        db = self.client['api_database']
        self.collection = db['models']

    def create(self, model_name: str, model):
        obj = {
            'model_name': model_name,
            'model': pickle.dumps(model),
        }
        try:
            self.collection.insert_one(obj)
            return True
        except DuplicateKeyError:
            return False

    def read(self, model_name: str):
        result = self.collection.find_one({'model_name': model_name})
        return result

    def update(self, model_name: str, model):
        filter_old_obj = {'model_name': model_name}
        new_obj = {'model': pickle.dumps(model)}
        print(new_obj)
        print({'$set': new_obj})
        self.collection.update_one(filter_old_obj, {'$set': new_obj})

    def delete(self, model_name: str):
        self.collection.delete_one({'model_name': model_name})

In [418]:
mdb = MongoDBClient('localhost', 27017)

In [419]:
mdb.create('model_1', LogisticRegression())

True

In [422]:
r = mdb.read('model_1')

In [424]:
pickle.loads(r['model'])

LogisticRegression()

In [425]:
r['model']

b'\x80\x04\x959\x01\x00\x00\x00\x00\x00\x00\x8c\x1esklearn.linear_model._logistic\x94\x8c\x12LogisticRegression\x94\x93\x94)\x81\x94}\x94(\x8c\x07penalty\x94\x8c\x02l2\x94\x8c\x04dual\x94\x89\x8c\x03tol\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x01C\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\rfit_intercept\x94\x88\x8c\x11intercept_scaling\x94K\x01\x8c\x0cclass_weight\x94N\x8c\x0crandom_state\x94N\x8c\x06solver\x94\x8c\x05lbfgs\x94\x8c\x08max_iter\x94Kd\x8c\x0bmulti_class\x94\x8c\x04auto\x94\x8c\x07verbose\x94K\x00\x8c\nwarm_start\x94\x89\x8c\x06n_jobs\x94N\x8c\x08l1_ratio\x94N\x8c\x10_sklearn_version\x94\x8c\x051.0.1\x94ub.'

In [436]:
pickle.dumps(1 + 's')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [489]:
pickle.loads(b'\x80\x04.')

UnpicklingError: unpickling stack underflow

In [672]:
import re
s = 'Курьер в сбeрмaркет'
s1 = translate(s, TranslateDirection.eng_rus)
re.findall(r'(sber.?market|сбер.?маркет)', s1, re.I)

['сбермаркет']

In [668]:
from enum import IntEnum

class TranslateDirection(IntEnum):
    eng_rus = 0
    rus_eng = 1


def translate(text_string: str, direction: TranslateDirection) -> str:
    """
    заменить похожие по написанию буквы
    """
    eng_chars = [ord(char) for char in 'EeTYyOoPpAaHKkXxCcBbM']
    rus_chars = [ord(char) for char in 'ЕеТУуОоРрАаНКкХхСсВьМ']
    tables = {
        TranslateDirection.eng_rus: dict(zip(eng_chars, rus_chars)),
        TranslateDirection.rus_eng: dict(zip(rus_chars, eng_chars)),
    }
    return text_string.translate(tables.get(direction, {}))

In [ ]:
translate()